In [49]:
import uuid
import requests

In [50]:
import dialogflow_v2beta1 as dialogflow

In [51]:
session_client = dialogflow.SessionsClient.from_service_account_json("banking-b287c-9bfcd2f6dec7.json")
project_id = "banking-b287c"

In [52]:
def detect_intent(session_id, text, language_code="en"):
    """Returns the result of detect intent with texts as inputs.

    Using the same `session_id` between requests allows continuation
    of the conversation."""
    
    session = session_client.session_path(project_id, session_id)

    text_input = dialogflow.types.TextInput(
        text=text, language_code=language_code)

    query_input = dialogflow.types.QueryInput(text=text_input)

    response = session_client.detect_intent(
        session=session, query_input=query_input)

    return response

In [80]:
def call_coco(component_id, session_id, **kwargs):
    return requests.post(
        "http://127.0.0.1:5000/api/exchange/"
        #"https://coco-235210.appspot.com/api/exchange/"
        f"{component_id}/{session_id}",
        json=kwargs, headers={"api-key":"master_key"}).json()

In [48]:
session_id = str(uuid.uuid4())
inputs = ["hi", "john smith", "yes", "I want to check my balance"]
for inp in inputs:
    response = detect_intent(session_id, inp).query_result
    print(f"U({response.intent.display_name}): {response.query_text}")
    print(f"B: {response.fulfillment_text}\n")

U(Default Welcome Intent): hi
B: Can you please tell me your name?

U(welcome.loop): john smith
B: Hello John. Nice to meet you. Is your last name smith? 

U(welcome.loop): yes
B: Nice to meet you, John smith. 

U(account.balance.check): I want to check my balance
B: What account do you want the balance for: savings or checking?



In [81]:
session_id = str(uuid.uuid4())

In [83]:
call_coco("register_vp3", session_id, user_input="chen@imperson.com")

{'component_done': False,
 'confidence': 0.5,
 'idontknow': False,
 'response': 'While I check the validity of chen@imperson.com, please choose a password. Any length, digits and case sensitive letters only.',
 'updated_context': {}}